1. Download hagmann connectome from TheVirtualBrain github
2. Unzip the package
3. Build NetworkX graph

In [ ]:
from shutil import unpack_archive
unpack_archive('./connectivity_998.zip', 'connectivity_998/')

In [ ]:
import numpy as np
# read nodes region names and centers
centres = np.genfromtxt('connectivity_998/centres.txt',
                        dtype='|U256,i,i,i',
                        encoding='utf-8',
                        names=['name','x','y','z'])

# convert nodes region names to dict
regions_dict = dict()
positions_dict = dict()
for idx, r in enumerate(centres):
    names_dict[idx] = r['name']
    positions_dict[idx] = (r['x'], r['y'],r['z'])
    
    
weights = np.genfromtxt('connectivity_998/weights.txt')
print(names_dict)

In [ ]:
import networkx as nx
graph = nx.Graph()
graphw = nx.from_numpy_matrix(weights)
graph.add_nodes_from(positions_dict)
graph.add_weighted_edges_from(graphw.edges())
#graph = nx.relabel_nodes(graph, names_dict)
#nx.set_node_attributes
print(graph.n())

In [ ]:
!cat connectivity_998/weights.txt

In [ ]:
!ls connectivity_998